**Conferir arquivo local**

In [8]:
# Só para ver se o Spark está ok
spark

In [9]:
import os

# Diretório onde o notebook está rodando (ex.: /home/jovyan/Trabalho-Final-PMD)
BASE_DIR = os.getcwd()
print("BASE_DIR:", BASE_DIR)

# Pasta onde estão os arquivos dentro do Jupyter
LOCAL_DADOS_DIR = os.path.join(BASE_DIR, "Dados")
print("LOCAL_DADOS_DIR:", LOCAL_DADOS_DIR)

# Conferir os arquivos nessa pasta
!ls "{LOCAL_DADOS_DIR}"

BASE_DIR: /mnt/notebooks/Trabalho-Final-PMD
LOCAL_DADOS_DIR: /mnt/notebooks/Trabalho-Final-PMD/Dados
dados_processos_cejusc_14052025.json  dados_processos_cejusc_14052025.parquet


**Definir caminhos (local e HDFS) e criar as camadas**

In [10]:
# Nome do arquivo parquet local
LOCAL_PARQUET_PATH = os.path.join(
    LOCAL_DADOS_DIR,
    "dados_processos_cejusc_14052025.parquet"
)

# URI para o Spark entender que é arquivo LOCAL (file://)
LOCAL_PARQUET_URI = "file://" + LOCAL_PARQUET_PATH

print("LOCAL_PARQUET_URI:", LOCAL_PARQUET_URI)

# Caminhos no HDFS (camadas do datalake)
HDFS_BRONZE_DIR = "/datalake/bronze"
HDFS_SILVER_DIR = "/datalake/silver"
HDFS_GOLD_DIR   = "/datalake/gold"

HDFS_BRONZE_PATH = "hdfs://" + HDFS_BRONZE_DIR + "/processos_cejusc"

print("HDFS_BRONZE_PATH:", HDFS_BRONZE_PATH)

LOCAL_PARQUET_URI: file:///mnt/notebooks/Trabalho-Final-PMD/Dados/dados_processos_cejusc_14052025.parquet
HDFS_BRONZE_PATH: hdfs:///datalake/bronze/processos_cejusc


In [11]:
# Criar as pastas das camadas no HDFS (se ainda não existirem)
!hdfs dfs -mkdir -p {HDFS_BRONZE_DIR}
!hdfs dfs -mkdir -p {HDFS_SILVER_DIR}
!hdfs dfs -mkdir -p {HDFS_GOLD_DIR}

# Conferir
!hdfs dfs -ls /datalake

Found 3 items
drwxr-xr-x   - root supergroup          0 2025-12-07 22:43 /datalake/bronze
drwxr-xr-x   - root supergroup          0 2025-12-07 22:43 /datalake/gold
drwxr-xr-x   - root supergroup          0 2025-12-07 22:43 /datalake/silver


**Ler o Parquet local (bronze) e gravar no HDFS**

In [12]:
# Ler o arquivo parquet local (dentro do container)
df_bronze = spark.read.parquet(LOCAL_PARQUET_URI)

df_bronze.printSchema()
df_bronze.show(5, truncate=False)
print("TOTAL REGISTROS (bronze):", df_bronze.count())

root
 |-- numero_processo: string (nullable = true)
 |-- comarca: string (nullable = true)
 |-- serventia: string (nullable = true)
 |-- assunto: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- polo_ativo: string (nullable = true)
 |-- polo_passivo: string (nullable = true)
 |-- cpf_cnpj_polo_ativo: string (nullable = true)
 |-- cpf_cnpj_polo_passivo: string (nullable = true)
 |-- inteiro_teor: string (nullable = true)
 |-- codg_cnj_audi: double (nullable = true)
 |-- codg_cnj_julgamento: double (nullable = true)
 |-- classificacao: string (nullable = true)

+-------------------------+---------------------+------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------

TOTAL REGISTROS (bronze): 16583


In [13]:
# Gravar no HDFS na camada BRONZE
(
    df_bronze
    .write
    .mode("overwrite")
    .parquet(HDFS_BRONZE_PATH)
)

# Conferir o que ficou no HDFS
!hdfs dfs -ls {HDFS_BRONZE_DIR}
!hdfs dfs -ls {HDFS_BRONZE_DIR}/processos_cejusc

Found 1 items
drwxr-xr-x   - root supergroup          0 2025-12-07 22:44 /datalake/bronze/processos_cejusc
Found 3 items
-rw-r--r--   2 root supergroup          0 2025-12-07 22:44 /datalake/bronze/processos_cejusc/_SUCCESS
-rw-r--r--   2 root supergroup       1425 2025-12-07 22:44 /datalake/bronze/processos_cejusc/part-00000-b80a3e98-d169-4e94-a673-1481049ace65-c000.snappy.parquet
-rw-r--r--   2 root supergroup   56527125 2025-12-07 22:44 /datalake/bronze/processos_cejusc/part-00002-b80a3e98-d169-4e94-a673-1481049ace65-c000.snappy.parquet
